# Dealing with errors after a run

In this example, we run the model on a list of three glaciers:
two of them will end with errors: one because it already failed at
preprocessing (i.e. prior to this run), and one during the run. We show how to recover from these errors and analyze them.

Some errors are also described in the OGGM documentation under [troubleshooting](https://docs.oggm.org/en/latest/faq.html?highlight=border#troubleshooting).

## The run

In [ ]:
# Libs
import pandas as pd
import xarray as xr

# Locals
import oggm.cfg as cfg
from oggm import utils, workflow, tasks

# Initialize OGGM and set up the default run parameters
cfg.initialize(logging_level='WARNING')

# Here we override some of the default parameters
# How many grid points around the glacier?
# We make it small because we want the model to error because
# of flowing out of the domain
cfg.PARAMS['border'] = 80

# This is the important bit!
# We tell OGGM to continue despite of errors
cfg.PARAMS['continue_on_error'] = True

# Local working directory (where OGGM will write its output)
WORKING_DIR = utils.gettempdir('OGGM_Errors')
utils.mkdir(WORKING_DIR, reset=True)
cfg.PATHS['working_dir'] = WORKING_DIR

rgi_ids = ['RGI60-11.00897', 'RGI60-11.01450', 'RGI60-11.03295']

# Go - get the pre-processed glacier directories
gdirs = workflow.init_glacier_directories(rgi_ids, from_prepro_level=4)

# We can step directly to the experiment!
# Random climate representative for the recent climate (1985-2015)
# with a negative bias added to the random temperature series
workflow.execute_entity_task(tasks.run_random_climate, gdirs,
                             nyears=150, seed=0,
                             temperature_bias=-1)

# Write the compiled output
utils.compile_glacier_statistics(gdirs);  # this is saved as glacier_statistics.csv in the WORKING_DIR folder
utils.compile_run_output(gdirs);  # this is saved as run_output.nc in the WORKING_DIR folder

## Show output

In [ ]:
# read aggregated output that is saved in WORKING_DIR
ds = xr.open_dataset(WORKING_DIR +'/run_output.nc')
pd_stats = pd.read_csv(WORKING_DIR+'/glacier_statistics.csv', index_col = 'rgi_id')

In [ ]:
# all possible statistics about the glaciers
pd_stats

- in the column *error_task* of *pd_stats*, we can see whether an error occurred, and if yes during which task
- *error_msg* describes the actual error message 

In [ ]:
pd_stats[['error_task', 'error_msg']]

We can also check which glacier failed at which task by using [compile_task_log]('https://docs.oggm.org/en/latest/generated/oggm.utils.compile_task_log.html#oggm.utils.compile_task_log').

In [ ]:
utils.compile_task_log(gdirs, task_names =['glacier_masks', 'compute_centerlines', 'flowline_model_run']) # this is saved as task_log.csv in the WORKING_DIR folder
# these are just example tasks that could be used to check for, others could be included as well

### RuntimeError occurred during task flowline_model_run on *RGI60-11.00897*

Glacier exceeds domain boundaries, at year: 98.08333333333333

- to remove this error just increase the domain boundary **before** doing `init_glacier_directories` ! Attention, this means that more data has to be downloaded and takes also much more time.

    The available options for `cfg.PARAMS['border']` are **10, 40, 80 or 160** at the moment; the unit is number of grid points outside the glacier boundaries. More about that in the OGGM documentation under [preprocessed files](https://docs.oggm.org/en/latest/input-data.html#pre-processed-directories).

In [ ]:
# reset to recompute statistics
utils.mkdir(WORKING_DIR, reset=True)
cfg.PATHS['working_dir'] = WORKING_DIR

# increase the surrounding amount of gridpoints
cfg.PARAMS['border'] = 160 
gdirs = workflow.init_glacier_directories(rgi_ids, from_prepro_level=4)
workflow.execute_entity_task(tasks.run_random_climate, gdirs,
                             nyears=150, seed=0,
                             temperature_bias=-1);

# recompute the output
# we can also get the run output directly from the methods
pd_stats = utils.compile_glacier_statistics(gdirs);  
ds = utils.compile_run_output(gdirs);  

In [ ]:
# check again
pd_stats[['error_task', 'error_msg']]


now *RGI60-11.00897* passes without errors!

## InvalidWorkflowError occurred during task run_random_climate on *RGI60-11.03295*
Need a valid `model_flowlines` file. If you explicitly want to use `inversion_flowlines`, set use_inversion_flowlines=True

- This error message is misleading and does not really describe the source of the error. Therefore we can look instead into the glacier_statistics via [compile_glacier_statistics](https://docs.oggm.org/en/latest/generated/oggm.utils.compile_glacier_statistics.html) or into the log output via [compile_task_log](https://docs.oggm.org/en/latest/generated/oggm.utils.compile_task_log.html#oggm.utils.compile_task_log)

In [ ]:
print('error_task: {}, error_msg: {}'.format(pd_stats.loc['RGI60-11.03295']['error_task'], 
                                             pd_stats.loc['RGI60-11.03295']['error_msg']))

Now we have a better understanding of the error for *RGI60-11.03295*: 
- OGGM can not work with this geometry of this glacier and could therefore not make a gridded mask of the glacier outlines. 
- there is no way to prevent this except you find a better way to pre-process the geometry of this glacier
- these glaciers have to be ignored! Less than 0.3% of glaciers globally have errors during the geometry processing or failures in computing certain topographical properties by e.g. invalid DEM, see [Sect. 4.2 Invalid Glaciers of the OGGM paper (Maussion et al., 2019)](https://gmd.copernicus.org/articles/12/909/2019/#section4)


## Ignoring those glaciers with errors that we can't solve!


- in the run_output, you can for example just use `*.dropna`:

In [ ]:
ds.dropna(dim='rgi_id')  # here we can e.g. find the volume evolution 

## What's next?

- read about [preprocessing errors](preprocessing_errors.ipynb)
- return to the [OGGM documentation](https://docs.oggm.org)
- back to the [table of contents](welcome.ipynb)